In [103]:
import pandas as pd

In [104]:
def getauthors(mytext):
    authors = set()
    for line in mytext:
        if len(line):
            if line[0].isalpha():
                comma_pos = line.find(",")
                authors.add(line[0:comma_pos])
    return authors

In [105]:
def getmodules(mytext):
    modules = set()
    for line in mytext:
        if len(line):
            if line[0].isnumeric() or line[0] == "-":
                first_tab = line.find("\t")
                second_tab = line.find("\t", first_tab + 1)
                modules.add(line[second_tab + 1:])
    return modules

In [106]:
def getmatrix(authors, modules):
    matrix = dict()
    for a in authors:
        matrix[a] = dict()
        for m in modules:
            matrix[a][m] = 0
    return matrix

In [107]:
def insertcommitstats(mytext, matrix):
    commitlines = list()
    for (i, line) in enumerate(mytext):
        if len(line):
            if line[0].isalpha():
                commitlines.append(i)
    endlines = list()
    for cl in commitlines[1:]:
        endlines.append(cl - 1)
    endlines.append(len(mytext))
    zipped_commits = list(zip(commitlines,endlines))
    for i in range(len(commitlines)):
        first_line = mytext[commitlines[i]]
        author_name = first_line[:first_line.find(",")]
        for j in range((commitlines[i] + 1),endlines[i]):
            first_tab = mytext[j].find("\t")
            second_tab = mytext[j].find("\t", first_tab + 1)
            if mytext[j][0] == "-":
                n_added = 0
                n_removed = 0
            else:
                n_added = int(mytext[j][:first_tab])
                n_removed = int(mytext[j][first_tab + 1:second_tab])
            module_name = mytext[j][second_tab + 1:]
            matrix[author_name][module_name] += n_added
            matrix[author_name][module_name] += n_removed

In [108]:
myfilepath = "myquery"
myfile = open(myfilepath)
mytext = [line for line in myfile]
myauthors = getauthors(mytext)
mymodules = getmodules(mytext)
mymatrix = getmatrix(myauthors, mymodules)
mymatrix_cp = mymatrix.copy()
insertcommitstats(mytext, mymatrix_cp)

In [109]:
mydf = pd.DataFrame(mymatrix_cp)

In [110]:
mydf.to_csv("mycsv.csv")